Import needed libraries

In [1]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool
import pyzx as zx
from pyzx import cnot_mapper, architecture # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.
from pyzx import circuit
from pyzx import *
from pyzx.parity_maps import CNOT_tracker, build_random_parity_map
from pyzx.linalg import Mat2
from pyzx.simplify import full_reduce

Generate random circuits

In [4]:
n_qubits = [4,5,6,7,8,9,10]
n_cnots = [2,4,5,8,10,15,20]
n_non_cnots = [0,1,2,4,5,8,10,15,20]
n_circuits = 100
np.random.seed(0)
GENERATION_ENABLED = True


comb_circuit_folder = "../circuits/combs/"
if GENERATION_ENABLED:
    for q in n_qubits:
        print(f"{q} Qubits")
        for c in n_cnots:
            print(f"{c} CNOTs")
            for nc in n_non_cnots:
                print(f"{nc} Non-CNOTs")
                # Create directory to store circuits of this type
                dest_folder = os.path.join(comb_circuit_folder, str(q)+"qubits", str(c)+"cnots", str(nc)+"non-cnots")
                os.makedirs(dest_folder, exist_ok=True)
                for i in range(n_circuits):
                    temp_circuit = Circuit(q)
                    build_random_parity_map(q, c, temp_circuit)
                    for j in range(nc):
                        non_cnot_gate = zx.circuit.NOT(np.random.randint(q)) # Generate gate on applied to random qubit
                        temp_circuit.gates.insert(np.random.randint(len(temp_circuit.gates)), non_cnot_gate) # Insert gate to random location of circuit
                    # Store circuit in qasm format
                    filename = f"Q-{q}_C-{c}_NC-{nc}_{i}.qasm"
                    dest_file = os.path.join(dest_folder, filename)
                    with open(dest_file, "w") as f:
                        f.write(temp_circuit.to_qasm())


4 Qubits
2 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
4 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
5 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
8 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
10 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
15 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
20 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
5 Qubits
2 CNOTs
0 Non-CNOTs
1 Non-CNOTs
2 Non-CNOTs
4 Non-CNOTs
5 Non-CNOTs
8 Non-CNOTs
10 Non-CNOTs
15 Non-CNOTs
20 Non-CNOTs
4 CNOTs
0 Non-CNOTs
1 Non-C

Define functions for comb based routing

In [17]:
def extract_sub_matrix(matrix, index_list):
    new_matrix = Mat2(np.zeros([matrix.rows(), len(index_list)], dtype=int))
    for row in range(new_matrix.rows()):
        for col in range(new_matrix.cols()):
            new_matrix.data[row][col] = matrix.data[row][index_list[col]]
    return new_matrix
def insert_sub_matrix(matrix, sub_matrix, index_list):
    for row in range(sub_matrix.rows()):
        for col in range(sub_matrix.cols()):
            matrix.data[row][index_list[col]] = sub_matrix.data[row][col]
    return matrix

In [18]:
def route_general_circuit(mode, circuit, arch, debug=False):
    # Decompose circuit
    comb_decomposition = CombDecomposition.from_circuit(circuit)
    comb = comb_decomposition.comb
    
    sub_circuits = []
    # Create a copy of the parity matrix of the comb to perform gaussian elimination on
    matrix = comb.matrix.copy()
    if debug:
        print("{}\n".format(matrix))
    # Clear the gates of the comb, as we will generate new ones as the elimination takes place
    comb.gates = []
    # Find initial qubits for the sub matrix
    qubits_in_matrix = []
    for qubit in range(comb.qubits):
        if qubit not in comb.holes.values():
            qubits_in_matrix.append(qubit)
    #print(qubits_in_matrix)
    # Generate sub matrix
    sub_matrix = extract_sub_matrix(matrix, qubits_in_matrix)
    # Apply gaussian elimation procedure of our choosing to sub matrix
    sub_circuits.append(cnot_mapper.gauss_return_circuit(mode,
                                     sub_matrix,
                                     architecture.create_architecture(arch, n_qubits=circuit.qubits)))
    sub_matrix = sub_circuits[-1].matrix
    if debug:
        print("{} :\n{}\n".format(qubits_in_matrix, sub_matrix))
    for gate in sub_circuits[-1].gates:
        # Add gates from sub circuit to comb circuit
        comb.gates.append(gate)
    # Place sub matrix back in large matrix
    insert_sub_matrix(matrix, Mat2.id(circuit.qubits), qubits_in_matrix)

    no_more_holes = len(comb.holes) == 0
    while not no_more_holes:
        # Update the qubits in the sub matrix to account for passing a hole
        for qubit_loc, qubit in enumerate(qubits_in_matrix):
            if qubit in comb.holes:
                qubits_in_matrix[qubit_loc] = comb.holes.pop(qubit)
        #print(qubits_in_matrix)
        if len(comb.holes) == 0:
            no_more_holes = True
        # Generate sub matrix
        sub_matrix = extract_sub_matrix(matrix, qubits_in_matrix)
        # Apply gaussian elimation procedure of our choosing to sub matrix
        sub_circuits.append(cnot_mapper.gauss_return_circuit(MODE,
                                         sub_matrix,
                                         architecture.create_architecture(arch, n_qubits=circuit.qubits)))
        sub_matrix = sub_circuits[-1].matrix
        if debug:
            print("{} :\n{}\n".format(qubits_in_matrix, sub_matrix))
        # Convert the gates of the sub matrix using the mapping
        for gate in sub_circuits[-1].gates:
            gate.control = qubits_in_matrix[gate.control]
            gate.target = qubits_in_matrix[gate.target]
            # Add gates from sub circuit to comb circuit
            comb.gates.append(gate)
        # Place sub matrix back in large matrix
        insert_sub_matrix(matrix, Mat2.id(circuit.qubits), qubits_in_matrix)
    # The comb has now been regenerated under the constraints of the hardware topology
    # Need to recombine the comb with its hole plugs to create a new circuit
    if debug:
        print("{}\n".format(matrix))
    if debug:
        return CombDecomposition.to_circuit(comb_decomposition), sub_circuits      
    else:
        return CombDecomposition.to_circuit(comb_decomposition)
    

Load comb circuits

In [19]:
circuit_properties = {"Qubits"    : 3,
                      "CNOTs"     : 2,
                      "Non-CNOTs" : 1}
path = os.path.join(comb_circuit_folder, 
                    str(circuit_properties["Qubits"])+"qubits", 
                    str(circuit_properties["CNOTs"])+"cnots", 
                    str(circuit_properties["Non-CNOTs"])+"non-cnots")
loaded_circuits = []
if os.path.exists(path):
    for file_name in os.listdir(path):
        loaded_circuits.append(circuit.Circuit.from_qasm_file(os.path.join(path, file_name)))
else:
    print("Directory Does Not Exist")

In [20]:
MODE = cnot_mapper.GAUSS_MODE
ARCHITECTURE = architecture.FULLY_CONNNECTED

for circ_num in range(len(loaded_circuits)):
    old_circ = loaded_circuits[circ_num]
    new_circ = route_general_circuit(MODE,
                                            old_circ.copy(),
                                            architecture.create_architecture(ARCHITECTURE, n_qubits=old_circ.qubits))
    print("Equal Circuits {} : {}".format(circ_num, old_circ.verify_equality(new_circ)))

IndexError: index 3 is out of bounds for axis 0 with size 3

Debug route_general_circuit

In [ ]:
MODE = cnot_mapper.STEINER_MODE
ARCHITECTURE = architecture.LINE

circ_num = 82
old_circ = loaded_circuits[circ_num]
new_circ, sub_circuits = route_general_circuit(MODE,
                                            old_circ.copy(),
                                            architecture.create_architecture(ARCHITECTURE, n_qubits=old_circ.qubits), debug=True)

from ipywidgets import widgets
from IPython.display import display, Markdown

COMB_ROUTING = True

def plotter(circ):
    print(f"Sub Circuit: {circ}")
    display(zx.draw(sub_circuits[circ]))
    #print(temp_CNOT_tracker.matrix)
    #new_circuit = route_general_circuit(MODE,
    #                                    sub_circuits[circ].copy(),
    #                                    architecture.create_architecture(ARCHITECTURE, n_qubits=sub_circuits[circ].qubits))
    #display(zx.draw(new_circuit))
    #print(new_temp_CNOT_tracker.matrix)
w = widgets.interactive(plotter, circ=(0,len(sub_circuits)-1))
slider = w.children[0]
slider.layout.width = "{!s}px".format(min(800,1000))
output = w.children[-1]
output.layout.height = "{!s}px".format(1000)
slider.value = 0
display(zx.draw(old_circ))
display(zx.draw(new_circ))
print(old_circ.verify_equality(new_circ))
w